<a href="https://colab.research.google.com/github/Sarend9/Class-Schedule/blob/Anton_Korneev/%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime

# Создаем соединение с базой данных
conn = sqlite3.connect("school_schedule.db")
cursor = conn.cursor()

# Создаем таблицу для календаря, если она не существует
cursor.execute('''
CREATE TABLE IF NOT EXISTS calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATE NOT NULL UNIQUE,
    weekday TEXT NOT NULL,
    is_weekend INTEGER NOT NULL CHECK (is_weekend IN (0, 1)),
    is_holiday INTEGER DEFAULT 0 CHECK (is_holiday IN (0, 1))
)
''')

# Создаем диапазон дат на 2025 год
dates = pd.date_range(start='2025-01-01', end='2025-12-31', freq='D')

# Создаем DataFrame
df = pd.DataFrame({
    'date': dates,
    'weekday': dates.dayofweek.map({
        0: 'Mon',  # Понедельник
        1: 'Tue',  # Вторник
        2: 'Wed',  # Среда
        3: 'Thu',  # Четверг
        4: 'Fri',  # Пятница
        5: 'Sat',  # Суббота
        6: 'Sun'   # Воскресенье
    }),
    'is_weekend': dates.dayofweek.isin([5, 6]).astype(int)
})

# Добавляем праздничные дни (пример для России)
holidays_2025 = [
    '2025-01-01', '2025-01-02', '2025-01-03', '2025-01-06', '2025-01-07',
    '2025-01-08', '2025-02-23', '2025-02-24', '2025-03-08', '2025-03-10',
    '2025-05-01', '2025-05-09', '2025-06-12', '2025-11-04'
]

df['is_holiday'] = df['date'].astype(str).isin(holidays_2025).astype(int)

# Заполняем базу данных
for _, row in df.iterrows():
    try:
        cursor.execute('''
        INSERT INTO calendar (date, weekday, is_weekend, is_holiday)
        VALUES (?, ?, ?, ?)
        ''', (
            row['date'].strftime('%Y-%m-%d'),
            row['weekday'],
            row['is_weekend'],
            row['is_holiday']
        ))
    except sqlite3.IntegrityError:
        # Пропускаем дубликаты дат
        continue

# Сохраняем изменения и закрываем соединение
conn.commit()
conn.close()

In [2]:
conn = sqlite3.connect("school_schedule.db")

# Создаем SQL-запрос для выборки всех данных из таблицы calendar
query = "SELECT * FROM calendar"

# Читаем данные в DataFrame
calendar_df = pd.read_sql(query, conn)

# Закрываем соединение
conn.close()

In [3]:
calendar_df

,id,date,weekday,is_weekend,is_holiday
0,1,2025-01-01,Wed,0,1
1,2,2025-01-02,Thu,0,1
2,3,2025-01-03,Fri,0,1
3,4,2025-01-04,Sat,1,0
4,5,2025-01-05,Sun,1,0
...,...,...,...,...,...
360,361,2025-12-27,Sat,1,0
361,362,2025-12-28,Sun,1,0
362,363,2025-12-29,Mon,0,0
363,364,2025-12-30,Tue,0,0


In [4]:
import sqlite3
from datetime import datetime
def create_schedule_table():
    conn = sqlite3.connect("school_schedule.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Schedule (
        lesson_id INTEGER PRIMARY KEY AUTOINCREMENT,
        class_id INTEGER NOT NULL,
        subject_id INTEGER NOT NULL,
        teacher_id INTEGER NOT NULL,
        room TEXT NOT NULL,
        start_time TEXT NOT NULL,
        end_time TEXT NOT NULL,
        weekday TEXT NOT NULL,
        valid_date DATE NOT NULL
    );
    """)
    conn.commit()
    conn.close()

import sqlite3

def fill_schedule():
    conn = sqlite3.connect("school_schedule.db")
    cursor = conn.cursor()

    # Расписание: (class_id, subject_id, teacher_id, room, start_time, end_time, weekday, valid_date)
    schedule_data = [
        # 7A класс — class_id = 1
        (1, 101, 201, "101А", "08:00", "08:45", "Mon", "5999-01-01"),
        (1, 102, 202, "101Б", "08:55", "09:40", "Mon", "5999-01-01"),
        (1, 103, 203, "101В", "10:00", "10:45", "Tue", "5999-01-01"),
        (1, 104, 204, "101Г", "11:00", "11:45", "Wed", "5999-01-01"),

        # 8B класс — class_id = 2
        (2, 105, 205, "202А", "08:00", "08:45", "Mon", "5999-01-01"),
        (2, 106, 206, "202Б", "08:55", "09:40", "Tue", "5999-01-01"),
        (2, 107, 207, "202В", "10:00", "10:45", "Wed", "5999-01-01"),
        (2, 108, 208, "202Г", "11:00", "11:45", "Thu", "5999-01-01"),

        # 9В класс — class_id = 3
        (3, 109, 209, "303А", "08:00", "08:45", "Tue", "5999-01-01"),
        (3, 110, 210, "303Б", "08:55", "09:40", "Wed", "5999-01-01"),
        (3, 111, 211, "303В", "10:00", "10:45", "Thu", "5999-01-01"),
        (3, 112, 212, "303Г", "11:00", "11:45", "Fri", "5999-01-01"),
    ]

    cursor.executemany("""
        INSERT INTO Schedule (
            class_id, subject_id, teacher_id, room, start_time, end_time, weekday, valid_date
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?);
    """, schedule_data)

    conn.commit()
    conn.close()
    print("Таблица Schedule успешно заполнена.")

In [5]:
create_schedule_table()
fill_schedule()

Таблица Schedule успешно заполнена.


In [6]:
conn = sqlite3.connect("school_schedule.db")

query = "SELECT * FROM Schedule"

# Читаем данные в DataFrame
Schedule_df = pd.read_sql(query, conn)
Schedule_df

,lesson_id,class_id,subject_id,teacher_id,room,start_time,end_time,weekday,valid_date
0,1,1,101,201,101А,08:00,08:45,Mon,5999-01-01
1,2,1,102,202,101Б,08:55,09:40,Mon,5999-01-01
2,3,1,103,203,101В,10:00,10:45,Tue,5999-01-01
3,4,1,104,204,101Г,11:00,11:45,Wed,5999-01-01
4,5,2,105,205,202А,08:00,08:45,Mon,5999-01-01
5,6,2,106,206,202Б,08:55,09:40,Tue,5999-01-01
6,7,2,107,207,202В,10:00,10:45,Wed,5999-01-01
7,8,2,108,208,202Г,11:00,11:45,Thu,5999-01-01
8,9,3,109,209,303А,08:00,08:45,Tue,5999-01-01
9,10,3,110,210,303Б,08:55,09:40,Wed,5999-01-01


In [7]:
import sqlite3

# Создаем соединение с базой данных
conn = sqlite3.connect("school_schedule.db")
cursor = conn.cursor()

# Создаем таблицу time_intervals, если она не существует
cursor.execute('''
drop TABLE IF EXISTS time_intervals
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS time_intervals (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    start_time TEXT NOT NULL UNIQUE,
    end_time TEXT NOT NULL
)
''')

# Очищаем таблицу перед заполнением (если нужно)
cursor.execute("DELETE FROM time_intervals")

# Генерируем временные интервалы
start_hour = 8
end_hour = 14
time_slots = []

for hour in range(start_hour, end_hour + 1):
    start = f"{hour:02d}:00"
    end = f"{hour:02d}:45"
    time_slots.append((start, end))

# Вставляем данные в таблицу
cursor.executemany('''
INSERT INTO time_intervals (start_time, end_time)
VALUES (?, ?)
''', time_slots)

# Сохраняем изменения и закрываем соединение
conn.commit()
conn.close()

print(f"Создано {len(time_slots)} временных интервалов с {start_hour}:00 до {end_hour}:45")

Создано 7 временных интервалов с 8:00 до 14:45


In [8]:
conn = sqlite3.connect("school_schedule.db")

query = "SELECT * FROM time_intervals"

# Читаем данные в DataFrame
time_intervals_df = pd.read_sql(query, conn)
time_intervals_df

,id,start_time,end_time
0,1,08:00,08:45
1,2,09:00,09:45
2,3,10:00,10:45
3,4,11:00,11:45
4,5,12:00,12:45
5,6,13:00,13:45
6,7,14:00,14:45


In [9]:
import sqlite3




def create_and_fill_subject_and_teacher_and_class_tables():
    conn = sqlite3.connect("school_schedule.db")
    cursor = conn.cursor()

    # Создание таблицы Subject
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Subject (
            subject_id INTEGER PRIMARY KEY,
            subject_name TEXT NOT NULL
        );
    """)

    # Создание таблицы Teacher
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Teacher (
            teacher_id INTEGER PRIMARY KEY,
            teacher_first_name TEXT NOT NULL,
            teacher_last_name TEXT NOT NULL,
            teacher_surname TEXT NOT NULL
        );
    """)
    # Создание таблицы Class
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Class (
            class_id INTEGER PRIMARY KEY,
            class_name TEXT NOT NULL
        );
    """)

    # Данные по предметам (пример)
    subjects = [
        (101, "Математика"),
        (102, "Русский язык"),
        (103, "Биология"),
        (104, "История"),
        (105, "Физика"),
        (106, "Химия"),
        (107, "География"),
        (108, "Литература"),
        (109, "Английский язык"),
        (110, "Информатика"),
        (111, "Физическая культура"),
        (112, "Музыка"),
    ]

    # Данные по учителям (пример)
    teachers = [
        (201, "Иван", "Петров", "Petrov"),
        (202, "Мария", "Иванова", "Ivanova"),
        (203, "Алексей", "Сидоров", "Sidorov"),
        (204, "Ольга", "Кузнецова", "Kuznetsova"),
        (205, "Дмитрий", "Смирнов", "Smirnov"),
        (206, "Елена", "Попова", "Popova"),
        (207, "Николай", "Волков", "Volkov"),
        (208, "Светлана", "Морозова", "Morozova"),
        (209, "Татьяна", "Зайцева", "Zaitseva"),
        (210, "Виктор", "Козлов", "Kozlov"),
        (211, "Ирина", "Новикова", "Novikova"),
        (212, "Павел", "Лебедев", "Lebedev"),
    ]

   # Данные: class_id, class_name
    class_data = [
        (1, "7А"),
        (2, "8Б"),
        (3, "9В")
    ]

    cursor.executemany("INSERT OR IGNORE INTO Class (class_id, class_name) VALUES (?, ?);", class_data)
    cursor.executemany("INSERT OR IGNORE INTO Subject (subject_id, subject_name) VALUES (?, ?);", subjects)
    cursor.executemany("""
        INSERT OR IGNORE INTO Teacher (teacher_id, teacher_first_name, teacher_last_name, teacher_surname)
        VALUES (?, ?, ?, ?);
    """, teachers)

    conn.commit()
    conn.close()
    print("Таблицы созданы и заполнены.")


In [10]:
create_and_fill_subject_and_teacher_and_class_tables()

Таблицы созданы и заполнены.


In [11]:
from datetime import datetime
import sqlite3

def deactivate_schedule_row(weekday, class_id, start_time):
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # текущие дата и время в строковом виде

    conn = sqlite3.connect("school_schedule.db")
    cursor = conn.cursor()

    cursor.execute("""
    UPDATE Schedule
    SET valid_date = ?
    WHERE weekday = ?
      AND class_id = ?
      AND start_time = ?
      AND valid_date = '5999-01-01';
    """, (now, weekday, class_id, start_time))

    conn.commit()
    conn.close()


In [12]:
def add_schedule_row(class_id, subject_id, teacher_id, room, start_time, end_time, weekday):
    conn = sqlite3.connect("school_schedule.db")
    cursor = conn.cursor()

    cursor.execute("""
    INSERT INTO Schedule (class_id, subject_id, teacher_id, room, start_time, end_time, weekday, valid_date)
    VALUES (?, ?, ?, ?, ?, ?, ?, '5999-01-01');
    """, (class_id, subject_id, teacher_id, room, start_time, end_time, weekday))

    conn.commit()
    conn.close()


In [13]:
add_schedule_row(3, 106, 204, '404А', '12:00', '12:45', 'Thu')

In [14]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Подключаемся к базе данных
conn = sqlite3.connect("school_schedule.db")

# Получаем текущую дату
today = datetime.now().date()

# Находим понедельник текущей недели
current_monday = today - timedelta(days=today.weekday())

# Вычисляем дату конца следующей недели (воскресенье)
end_date = current_monday + timedelta(days=13)  # 2 недели = 14 дней -1
cursor = conn.cursor()

# Создаем таблицу для календаря, если она не существует
cursor.execute('''
drop table if exists current_period
''')
# Формируем SQL-запрос
query = """
SELECT *
FROM calendar
WHERE date BETWEEN ? AND ?
ORDER BY date
"""

# Выполняем запрос
df = pd.read_sql(query, conn, params=(current_monday.strftime('%Y-%m-%d'),
                                end_date.strftime('%Y-%m-%d')))


df.to_sql('current_period', conn, index=False)



conn = sqlite3.connect("school_schedule.db")

cursor = conn.cursor()

# Создаем таблицу для календаря, если она не существует
cursor.execute('''
drop table if exists current_period_class
''')
cursor.execute('''
create table current_period_class as SELECT c.class_name, cp.* FROM current_period as cp cross join Class as c order by 1, 3
''')

query = " select * from current_period_class"

# Читаем данные в DataFrame
current_interval_class_df = pd.read_sql(query, conn)
current_interval_class_df

cursor.execute('''
drop table if exists current_period_class_intervals
''')
cursor.execute('''
create table current_period_class_intervals as SELECT  cp.*, ti.start_time, ti.end_time
FROM current_period_class as cp cross join time_intervals as ti order by 1, 3, ti.start_time
''')

query = "SELECT * FROM current_period_class_intervals order by 1, 3"

# Читаем данные в DataFrame
current_period_class_intervalss_df = pd.read_sql(query, conn)
current_period_class_intervalss_df

,class_name,id,date,weekday,is_weekend,is_holiday,start_time,end_time
0,7А,160,2025-06-09,Mon,0,0,08:00,08:45
1,7А,160,2025-06-09,Mon,0,0,09:00,09:45
2,7А,160,2025-06-09,Mon,0,0,10:00,10:45
3,7А,160,2025-06-09,Mon,0,0,11:00,11:45
4,7А,160,2025-06-09,Mon,0,0,12:00,12:45
...,...,...,...,...,...,...,...,...
289,9В,173,2025-06-22,Sun,1,0,10:00,10:45
290,9В,173,2025-06-22,Sun,1,0,11:00,11:45
291,9В,173,2025-06-22,Sun,1,0,12:00,12:45
292,9В,173,2025-06-22,Sun,1,0,13:00,13:45


In [15]:
query = """
SELECT
    cpc.class_name,
    cpc.date,
    cpc.weekday,
    cpc.start_time,
    cpc.end_time,
    COALESCE(s.subject_name, '') AS subject_name,
    COALESCE(t.teacher_last_name, '') AS teacher_last_name,
    COALESCE(sc.room, '') AS room
FROM current_period_class_intervals cpc
LEFT JOIN Class c ON cpc.class_name = c.class_name
LEFT JOIN (
    SELECT DISTINCT
        class_id,
        weekday,
        start_time,
        subject_id,
        teacher_id,
        room
    FROM Schedule
    where valid_date >= '5999-01-01'
) sc ON cpc.weekday = sc.weekday
    AND c.class_id = sc.class_id
    AND cpc.start_time = sc.start_time
LEFT JOIN Subject s ON sc.subject_id = s.subject_id
LEFT JOIN Teacher t ON sc.teacher_id = t.teacher_id
ORDER BY cpc.class_name, cpc.date, cpc.start_time
"""

total = pd.read_sql(query, conn)
total[total['room']!='']

,class_name,date,weekday,start_time,end_time,subject_name,teacher_last_name,room
0,7А,2025-06-09,Mon,08:00,08:45,Математика,Петров,101А
9,7А,2025-06-10,Tue,10:00,10:45,Биология,Сидоров,101В
17,7А,2025-06-11,Wed,11:00,11:45,История,Кузнецова,101Г
49,7А,2025-06-16,Mon,08:00,08:45,Математика,Петров,101А
58,7А,2025-06-17,Tue,10:00,10:45,Биология,Сидоров,101В
66,7А,2025-06-18,Wed,11:00,11:45,История,Кузнецова,101Г
98,8Б,2025-06-09,Mon,08:00,08:45,Физика,Смирнов,202А
114,8Б,2025-06-11,Wed,10:00,10:45,География,Волков,202В
122,8Б,2025-06-12,Thu,11:00,11:45,Литература,Морозова,202Г
147,8Б,2025-06-16,Mon,08:00,08:45,Физика,Смирнов,202А


In [21]:
from ipywidgets import interact, widgets
from IPython.display import display, Markdown

# Загрузка данных
df = total
df["date"] = pd.to_datetime(df["date"])

# Функция расписания
def get_schedule_for_class(df, class_name):
    class_df = df[df["class_name"] == class_name].copy()
    class_df = class_df.sort_values(by=["date", "start_time"])
    grouped = class_df.groupby(["date", "weekday"])

    output = ""
    for (date, weekday), group in grouped:
        output += f"### 📅 {weekday}, {date.strftime('%d.%m.%Y')} (класс {class_name})\n"
        for i, row in enumerate(group.itertuples(), start=1):
            subject = row.subject_name if row.subject_name!='' else "❌ Нет урока"
            teacher = f" ({row.teacher_last_name})" if pd.notna(row.teacher_last_name) else ""
            room = f", каб. {row.room}" if pd.notna(row.room) else ""
            output += f"{i}. {row.start_time}–{row.end_time} — {subject}{teacher}{room}\n"
        output += "\n"
    display(Markdown(output))

# Виджет
interact(
    get_schedule_for_class,
    df=widgets.fixed(df),
    class_name=widgets.Dropdown(options=sorted(df["class_name"].unique()), description='Класс:')
)

interactive(children=(Dropdown(description='Класс:', options=('7А', '8Б', '9В'), value='7А'), Output()), _dom_…

<function __main__.get_schedule_for_class(df, class_name)>

In [20]:
subject

NameError: name 'subject' is not defined

In [ ]:
matrix_v2 = []

# Первая строка с class_name и остальными заголовками
header = ['date', 'weekday', 'start_time', 'end_time', 'subject_name', 'teacher_last_name', 'room']
matrix_v2.append(['class_name'] + header)

# Добавляем данные
for _, row in total.iterrows():
    matrix_v2.append([
        row['class_name'],
        row['date'],
        row['weekday'],
        row['start_time'],
        row['end_time'],
        row['subject_name'],
        row['teacher_last_name'],
        row['room']
    ])


In [ ]:
matrix_v2.style.set_properties(**{'text-align': 'left'})

AttributeError: 'list' object has no attribute 'style'

In [ ]:
query = "SELECT cpc.class_name, cpc.date, cpc.weekday, cpc.start_time, cpc.end_time, s.subject_name, t.teacher_last_name, sc.room FROM current_period_class_intervals cpc left join Class c on cpc.class_name = c.class_name left join Schedule sc on cpc.weekday = sc.weekday and c.class_id = sc.class_id and sc.start_time = cpc.start_time left join Subject s on sc.subject_id = s.subject_id left join Teacher t on t.teacher_id = sc.teacher_id order by 1, 3"

# Читаем данные в DataFrame
total = pd.read_sql(query, conn)
total = total.fillna('')
total.drop_duplicates()

,class_name,date,weekday,start_time,end_time,subject_name,teacher_last_name,room
0,7А,2025-06-13,Fri,08:00,08:45,,,
1,7А,2025-06-13,Fri,09:00,09:45,,,
2,7А,2025-06-13,Fri,10:00,10:45,,,
3,7А,2025-06-13,Fri,11:00,11:45,,,
4,7А,2025-06-13,Fri,12:00,12:45,,,
...,...,...,...,...,...,...,...,...
325,9В,2025-06-18,Wed,10:00,10:45,,,
326,9В,2025-06-18,Wed,11:00,11:45,,,
327,9В,2025-06-18,Wed,12:00,12:45,,,
328,9В,2025-06-18,Wed,13:00,13:45,,,


In [ ]:
total = total.fillna('')
